In [ ]:
### google play reviews
import json
import re
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import Sort, reviews, app

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

In [ ]:
app_packages = [
  'com.navyfederal.android'
]

In [ ]:
app_infos = []

for ap in tqdm(app_packages):
    info = app(ap, lang='en', country='us')
    #del info['comments']
    app_infos.append(info)


In [ ]:
app_reviews = []

for ap in tqdm(app_packages):
    for score in list(range(1, 6)):
        rvs, _ = reviews(ap,lang='en',country='us',count= 20000000,filter_score_with=score)
        print(rvs)
        for r in rvs:
            reviewId = r.get('reviewId')
            userName = r.get('userName')
            content = str(r.get('content'))
            content = re.sub(r'[^a-zA-Z0-9]',' ',content)
            score = r.get('score')
            thumbsUpCount = r.get('thumbsUpCount')
            at = r.get('at')
            replyContent = r.get('replyContent')
            if replyContent is None:
                replyContent =''
            else:
                replyContent = re.sub(r'[^a-zA-Z0-9]',' ',replyContent)
            app_reviews.append((reviewId, userName, content, score, thumbsUpCount, at, replyContent))

In [ ]:
print(len(app_reviews))

In [ ]:
# app_reviews = []

# for ap in tqdm(app_packages):
#     for score in list(range(1, 6)):
#         for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
#             rvs, _ = reviews(ap,lang='en',country='us',sort=sort_order,count= 200 if score == 3 else 100,filter_score_with=score)
#             for r in rvs:
#                 r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
#                 r['appId'] = ap
#                 app_reviews.extend(rvs)
                

In [ ]:
app_reviews_df = pd.DataFrame(app_reviews, columns=['reviewId', 'userName', 'content', 'score', 'thumbsUpCount', 'at', 'replyContent'])



In [ ]:
def removeNone(app_reviews_df):
    mask = app_reviews_df.applymap(lambda x: x is None)
    cols = app_reviews_df.columns[(mask).any()]
    for col in app_reviews_df[cols]:
        app_reviews_df.loc[mask[col], col] = ''
    return app_reviews_df

app_reviews_df = removeNone(app_reviews_df)

In [ ]:
def whitespace_remover(dataframe):
    
    # iterating over the columns
    for i in dataframe.columns:
          
        # checking datatype of each columns
        if dataframe[i].dtype == 'object':
              
            # applying strip function on column
            dataframe[i] = dataframe[i].map(str.strip)
        else:
              
            # if condn. is False then it will do nothing.
            pass
    return dataframe
  
# applying whitespace_remover function on dataframe
app_reviews_df = whitespace_remover(app_reviews_df)
  
# printing dataframe
display(app_reviews_df)

In [ ]:
import glob
files_present = glob.glob('/Volumes/Navya/UB/navyfederal/Reviews/google_reviews.csv')
print(files_present)

In [ ]:
if not files_present:
    app_reviews_df.to_csv('/Volumes/Navya/UB/navyfederal/Reviews/google_reviews.csv', index=None, header=True)
else:
    print("File already Exists!!!")